In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import torch
import numpy as np
import matplotlib.pyplot as plt

import implicitmodules.torch as dm

In [ ]:
def rot(theta):
    return torch.tensor([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

width = 25.
height = 3.
nx = 80
ny = 8
x, y = torch.meshgrid([torch.linspace(-width/2, width/2, nx), torch.linspace(-height/2, height/2, ny)])
pts_sil = dm.Utilities.grid2vec(x, y)
ps = torch.zeros_like(pts_sil)

width1 = 20.
height1 = 2.
nx1 = 30
ny1 = 3
x1, y1 = torch.meshgrid([torch.linspace(-width1/2, width1/2, nx1), torch.linspace(-height1/2, height1/2, ny1)])
pts = dm.Utilities.grid2vec(x1, y1)
R = rot(0.).repeat(pts.shape[0], 1, 1)
p1 = torch.zeros_like(pts)
p1[0, 0] = -15000.

C = torch.zeros(pts.shape[0], 2, 1)
C[:, 0] = pts[:, 1].view(-1, 1)*torch.cos(np.pi*pts[:, 0].view(-1, 1)/width1*2*4)

sigma = 1.1
nu = 0.

model = dm.DeformationModules.ImplicitModule1(2, pts.shape[0], sigma, C, nu)
model.manifold.fill_gd((pts.clone().requires_grad_(), R.clone().requires_grad_()))
model.manifold.fill_cotan((p1.clone().requires_grad_(), torch.zeros_like(R).requires_grad_()))

sil = dm.DeformationModules.SilentLandmarks(2, pts_sil.shape[0])
sil.manifold.fill_gd(pts_sil.clone().requires_grad_())
sil.manifold.fill_cotan(ps.clone().requires_grad_())

aabb = dm.Utilities.AABB.build_from_points(torch.cat([compound[0].manifold.gd.detach(), pts_sil], dim=0)).squared().scale(1.2)

grid = dm.DeformationModules.DeformationGrid(aabb, [128, 128])

compound = dm.DeformationModules.CompoundModule([sil, model, grid])
out = dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian(compound), it=10, method='euler')

In [15]:
%matplotlib qt5
ax = plt.subplot(1, 2, 1)
plt.plot(pts_sil.numpy()[:, 0], pts_sil.numpy()[:, 1], '.')
plt.plot(pts.numpy()[:, 0], pts.numpy()[:, 1], 'x')
x, y = dm.Utilities.vec2grid(pts_sil, nx, ny)
dm.Utilities.plot_grid(ax, x.detach().numpy(), y.detach().numpy(), color='C0')
plt.axis(aabb.totuple())

ax1 = plt.subplot(1, 2, 2)
plt.plot(compound[0].manifold.gd.detach().numpy()[:, 0], compound[0].manifold.gd.detach().numpy()[:, 1], '.')
plt.plot(compound[1].manifold.gd[0].detach().numpy()[:, 0], compound[1].manifold.gd[0].detach().numpy()[:, 1], 'x')
x, y = dm.Utilities.vec2grid(compound[0].manifold.gd, nx, ny)
dm.Utilities.plot_grid(ax1, x.detach().numpy(), y.detach().numpy(), color='C0')

dm.Utilities.plot_grid(ax1, grid.togrid()[0], grid.togrid()[1], color='xkcd:light blue', lw=0.3)

plt.axis(aabb.totuple())
plt.show()